In [ ]:
import json
import boto3
import os

sns = boto3.client('sns')

SNS_TOPIC_ARN = os.environ.get('SNS_TOPIC_ARN', 'arn:aws:sns:us-east-1:123456789012:irrigacao-alertas')

def lambda_handler(event, context):
    # Se vier JSON no body (API Gateway HTTP API)
    try:
        body = event.get("body")
        if isinstance(body, str):
            data = json.loads(body)
        elif isinstance(body, dict):
            data = body
        else:
            data = {}
    except Exception:
        data = {}

    umidade = data.get("umidade")
    mensagem_custom = data.get("mensagem", "Alerta do sistema de irrigação!")

    mensagem = f"{mensagem_custom} Umidade do solo: {umidade}%" if umidade is not None else mensagem_custom

    try:
        response = sns.publish(
            TopicArn=SNS_TOPIC_ARN,
            Message=mensagem,
            Subject="Alerta de Irrigação"
        )
        return {
            "statusCode": 200,
            "body": json.dumps({"status": "ok", "messageId": response.get("MessageId")})
        }
    except Exception as e:
        return {
            "statusCode": 500,
            "body": json.dumps({"status": "erro", "detalhe": str(e)})
        }
